In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from adam_prepare import titanic_pipeline

Let's read in our data from the titanic pipeline function!

In [3]:
train, val, test = titanic_pipeline()
train.shape, val.shape, test.shape

((623, 9), (134, 9), (134, 9))

We need to create our X and y subsets.

Before modeling, it's always important to define a baseline!

Now we are ready to create a [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) object and start modeling!